In [3]:
"""
cv2.face 에러날 경우 아래 명령어 실행
    pip install opencv-contrib-python
    위 명령어 실행시 에러날 경우 : https://potensj.tistory.com/73

※ 참고 사이트
- 얼굴 인식하여 잠금 해제
    https://blog.naver.com/chandong83/221436424539
    
"""

'#개발 '

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [3]:
data_path = './98_LearningData/04_StudyData/1170120/'

In [4]:
#faces폴더에 있는 파일 리스트 얻기 
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

In [5]:
#데이터와 매칭될 라벨 변수 
Training_Data, Labels = [], []

In [6]:
#파일 개수 만큼 루프 
for i, files in enumerate(onlyfiles):    
    image_path = data_path + onlyfiles[i]
    #이미지 불러오기 
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    #이미지 파일이 아니거나 못 읽어 왔다면 무시
    if images is None:
        continue    
    #Training_Data 리스트에 이미지를 바이트 배열로 추가 
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    #Labels 리스트엔 카운트 번호 추가 
    Labels.append(i)

In [7]:
#훈련할 데이터가 없다면 종료.
if len(Labels) == 0:
    print("There is no data to train.")
    exit()

In [8]:
#Labels를 32비트 정수로 변환
Labels = np.asarray(Labels, dtype=np.int32)
#모델 생성 
model = cv2.face.LBPHFaceRecognizer_create()
#학습 시작 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model Training Complete!!!!!")

AttributeError: module 'cv2.cv2' has no attribute 'face'